<a href="https://colab.research.google.com/github/alamwasim/job-a-thon/blob/main/JOB_A_THON_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
train=pd.read_csv("/content/gdrive/MyDrive/ml_data/jobathon/jobathon_train.csv")
test=pd.read_csv("/content/gdrive/MyDrive/ml_data/jobathon/jobathon_test.csv")

In [4]:
train.head()

,ID,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium,Response
0,1,C3,3213,Rented,Individual,36,36,No,X1,14+,3.0,22,11628.0,0
1,2,C5,1117,Owned,Joint,75,22,No,X2,NaN,NaN,22,30510.0,0
2,3,C5,3732,Owned,Individual,32,32,No,NaN,1.0,1.0,19,7450.0,1
3,4,C24,4378,Owned,Joint,52,48,No,X1,14+,3.0,19,17780.0,0
4,5,C8,2190,Rented,Individual,44,44,No,X2,3.0,1.0,16,10404.0,0


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50882 entries, 0 to 50881
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID                       50882 non-null  int64  
 1   City_Code                50882 non-null  object 
 2   Region_Code              50882 non-null  int64  
 3   Accomodation_Type        50882 non-null  object 
 4   Reco_Insurance_Type      50882 non-null  object 
 5   Upper_Age                50882 non-null  int64  
 6   Lower_Age                50882 non-null  int64  
 7   Is_Spouse                50882 non-null  object 
 8   Health Indicator         39191 non-null  object 
 9   Holding_Policy_Duration  30631 non-null  object 
 10  Holding_Policy_Type      30631 non-null  float64
 11  Reco_Policy_Cat          50882 non-null  int64  
 12  Reco_Policy_Premium      50882 non-null  float64
 13  Response                 50882 non-null  int64  
dtypes: float64(2), int64(6

In [6]:
#Replacing Missing Values with 'null' string. These all wil be treated as categorical data
train[['Health Indicator','Holding_Policy_Type']]=train[['Health Indicator','Holding_Policy_Type']].fillna("null")
test[['Health Indicator','Holding_Policy_Type']]=test[['Health Indicator','Holding_Policy_Type']].fillna("null")
#Replacing Null with 0 in Holding policy Duration
train[['Holding_Policy_Duration']]=train[['Holding_Policy_Duration']].fillna(0)
test[['Holding_Policy_Duration']]=test[['Holding_Policy_Duration']].fillna(0)
#Replacing 14+ as 15 in Holding policy Duration
train.loc[train.Holding_Policy_Duration=='14+','Holding_Policy_Duration']=15
test.loc[test.Holding_Policy_Duration=='14+','Holding_Policy_Duration']=15

In [7]:
#Creating new features 'Is_Holding_Policy' to indicate if particular ID holds any Policy or not base upon missing values in Holding Policy Type
train.loc[:,'Is_Holding_Policy']=1
train.loc[train.Holding_Policy_Type=="null",'Is_Holding_Policy']=0
test.loc[:,'Is_Holding_Policy']=1
test.loc[test.Holding_Policy_Type=="null",'Is_Holding_Policy']=0
#Creating new features 'mean_age' mean_age based upon lower and upper age
train.loc[:,'mean_age']=(train.Upper_Age+train.Lower_Age)/2
test.loc[:,'mean_age']=(test.Upper_Age+test.Lower_Age)/2
#Age Gap
train.loc[:,'age_gap']=train.Upper_Age-train.Lower_Age
test.loc[:,'age_gap']=test.Upper_Age-test.Lower_Age

In [8]:
new_var_in_test=[]
new_var_in_train=[]
for c in train.select_dtypes('object').columns:
  new_var_in_test.append(len(set(test[c].values)-set(train[c].values)))
  new_var_in_train.append(len(set(train[c].values)-set(test[c].values)))
pd.DataFrame({'new_var_in_test':new_var_in_test,'new_var_in_train':new_var_in_train},index=train.select_dtypes('object').columns)

,new_var_in_test,new_var_in_train
City_Code,0,0
Accomodation_Type,0,0
Reco_Insurance_Type,0,0
Is_Spouse,0,0
Health Indicator,0,0
Holding_Policy_Duration,0,0
Holding_Policy_Type,0,0


In [9]:
train.Holding_Policy_Duration=train.Holding_Policy_Duration.astype('float32').astype('int32')
test.Holding_Policy_Duration=test.Holding_Policy_Duration.astype('float32').astype('int32')

In [10]:
le_col=['City_Code', 'Accomodation_Type', 'Reco_Insurance_Type', 'Is_Spouse',
       'Health Indicator', 'Holding_Policy_Type']
def le_fit(df,col):
  le_dict={}
  for c in col:
    le_dict[c]=dict(zip(train[c].unique(),range(train[c].nunique())))
  print("le_dict :",le_dict)
  return le_dict
def le_transform(df,col):
  for c in col:
    df[c]=df[c].map(le_dict[c],na_action='ignore').fillna(-1)
le_dict=le_fit(train,le_col)
le_transform(train,le_col)
le_transform(test,le_col)

le_dict : {'City_Code': {'C3': 0, 'C5': 1, 'C24': 2, 'C8': 3, 'C9': 4, 'C1': 5, 'C15': 6, 'C28': 7, 'C27': 8, 'C7': 9, 'C20': 10, 'C25': 11, 'C4': 12, 'C2': 13, 'C34': 14, 'C10': 15, 'C17': 16, 'C18': 17, 'C16': 18, 'C29': 19, 'C33': 20, 'C26': 21, 'C19': 22, 'C6': 23, 'C12': 24, 'C13': 25, 'C11': 26, 'C14': 27, 'C22': 28, 'C23': 29, 'C21': 30, 'C36': 31, 'C32': 32, 'C30': 33, 'C35': 34, 'C31': 35}, 'Accomodation_Type': {'Rented': 0, 'Owned': 1}, 'Reco_Insurance_Type': {'Individual': 0, 'Joint': 1}, 'Is_Spouse': {'No': 0, 'Yes': 1}, 'Health Indicator': {'X1': 0, 'X2': 1, 'null': 2, 'X4': 3, 'X3': 4, 'X6': 5, 'X5': 6, 'X8': 7, 'X7': 8, 'X9': 9}, 'Holding_Policy_Type': {3.0: 0, 'null': 1, 1.0: 2, 4.0: 3, 2.0: 4}}


In [11]:
train.select_dtypes('object').columns

Index([], dtype='object')

In [12]:
train.columns

Index(['ID', 'City_Code', 'Region_Code', 'Accomodation_Type',
       'Reco_Insurance_Type', 'Upper_Age', 'Lower_Age', 'Is_Spouse',
       'Health Indicator', 'Holding_Policy_Duration', 'Holding_Policy_Type',
       'Reco_Policy_Cat', 'Reco_Policy_Premium', 'Response',
       'Is_Holding_Policy', 'mean_age', 'age_gap'],
      dtype='object')

In [75]:
from sklearn.preprocessing import OneHotEncoder
ohe_col=['Health Indicator', 'Holding_Policy_Duration', 'Holding_Policy_Type','Reco_Policy_Cat']
enc = OneHotEncoder(handle_unknown='ignore',categories=ohe_col)
enc.fit(train)
train=enc.trandform(train)
test=enc.transform(test)

ValueError: ignored

In [ ]:
[]

In [52]:
sel_col=['City_Code', 'Region_Code', 'Accomodation_Type',
       'Reco_Insurance_Type', 'Upper_Age', 'Lower_Age', 'Is_Spouse',
       'Health Indicator', 'Holding_Policy_Duration', 'Holding_Policy_Type',
       'Reco_Policy_Cat', 'Reco_Policy_Premium',
       'Is_Holding_Policy', 'mean_age', 'age_gap']
target_col=['Response']

In [63]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(train[sel_col])
train[sel_col]=sc.transform(train[sel_col])
test[sel_col]=sc.transform(test[sel_col])

In [64]:
X, x_test, y, y_test = train_test_split(train[sel_col], train[target_col], test_size=0.2, random_state=786)
print(X.shape,x_test.shape,train.shape,X.shape[0]+x_test.shape[0])

(40705, 15) (10177, 15) (50882, 17) 50882


In [60]:
import tensorflow as tf
from tensorflow import keras

In [71]:
# model = Sequential()
# model.add(Dense(10, input_dim=X.shape[1], activation='relu', kernel_initializer='he_normal'))
# model.add(Dense(10, input_dim=X.shape[1], activation='relu', kernel_initializer='he_normal'))
# model.add(Dense(1, activation='sigmoid'))
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(X.shape[1],)),
    keras.layers.Dense(10, activation=tf.nn.relu),
    keras.layers.Dense(5, activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid),
])
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.AUC()])
# fit the keras model on the dataset
model.fit(X, y, epochs=10, batch_size=15, verbose=2)
# evaluate the keras model
_, accuracy = model.evaluate(x_test, y_test, verbose=0)
roc_auc_score(y,model.predict(X))

Epoch 1/10
2714/2714 - 3s - loss: 0.5490 - auc_3: 0.5514
Epoch 2/10
2714/2714 - 3s - loss: 0.5410 - auc_3: 0.5660
Epoch 3/10
2714/2714 - 3s - loss: 0.5393 - auc_3: 0.5743
Epoch 4/10
2714/2714 - 3s - loss: 0.5387 - auc_3: 0.5752
Epoch 5/10
2714/2714 - 3s - loss: 0.5385 - auc_3: 0.5791
Epoch 6/10
2714/2714 - 3s - loss: 0.5381 - auc_3: 0.5806
Epoch 7/10
2714/2714 - 3s - loss: 0.5380 - auc_3: 0.5809
Epoch 8/10
2714/2714 - 3s - loss: 0.5378 - auc_3: 0.5820
Epoch 9/10
2714/2714 - 3s - loss: 0.5375 - auc_3: 0.5835
Epoch 10/10
2714/2714 - 3s - loss: 0.5373 - auc_3: 0.5850


0.5885917646194967

In [72]:
roc_auc_score(y_test,model.predict(x_test))
# 0.5622076813985002

0.5699944187504734

In [73]:
model.summary()

Model: "sequential_30"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 15)                0         
_________________________________________________________________
dense_60 (Dense)             (None, 10)                160       
_________________________________________________________________
dense_61 (Dense)             (None, 5)                 55        
_________________________________________________________________
dense_62 (Dense)             (None, 1)                 6         
Total params: 221
Trainable params: 221
Non-trainable params: 0
_________________________________________________________________


In [41]:
pred=model.predict(test).ravel()

In [42]:
submission=pd.DataFrame({'ID':test.ID,'Response':pred})

In [44]:
submission.Response.value_counts()

1.0    21805
Name: Response, dtype: int64

In [49]:
test_pred=model.predict(x_test).ravel()

In [50]:
x_test['pred']=test_pred

,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium,Response,Is_Holding_Policy,mean_age,age_gap,pred
3918,9,196,1,0,49,49,0,2,4,4,13,14742.0,0,1,49.0,0,8.578342e-11
37480,23,322,1,1,72,71,1,3,15,0,9,30189.6,0,1,71.5,1,2.196536e-13
10936,3,2141,0,0,26,26,0,0,0,1,9,9872.0,1,0,26.0,0,1.000000e+00
46021,18,1079,0,0,23,23,0,4,0,1,1,6948.0,0,0,23.0,0,3.360103e-07
18314,21,772,1,0,53,53,0,1,15,0,12,10920.0,0,1,53.0,0,1.372899e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26049,1,1240,0,0,53,53,0,2,6,4,15,13002.0,0,1,53.0,0,2.722201e-10
43510,13,706,0,0,21,21,0,1,1,3,13,7524.0,1,1,21.0,0,1.000000e+00
32481,6,213,0,0,29,29,0,1,5,0,12,9044.0,0,1,29.0,0,9.971591e-07
13175,3,349,0,0,44,44,0,4,13,0,21,12900.0,0,1,44.0,0,3.663321e-08
